# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [4]:
import pandas as pd
import plotly.express as px
import numpy as np
from plotly import graph_objects as go

# Notebook Presentation

In [5]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [6]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [7]:
df_apps.shape

(10841, 12)

In [8]:
df_apps.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')

In [110]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
9046,AT&T Visual Voicemail,COMMUNICATION,3.70,13761,3.50,"10,000,000",Free,0,Everyone,Communication
10284,Granny,GAME,4.50,1137267,59.00,"50,000,000",Free,0,Teen,Arcade
1912,Restaurantführer Südbaden,TRAVEL_AND_LOCAL,4.10,46,12.00,"1,000",Free,0,Everyone,Travel & Local
9712,Camera for Android,PHOTOGRAPHY,4.20,240475,9.50,"10,000,000",Free,0,Everyone,Photography
8234,MetaTrader 4,FINANCE,4.60,260547,9.15,"5,000,000",Free,0,Everyone,Finance


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns. 

In [10]:
df2_apps = pd.concat([df_apps.pop(x) for x in ['Last_Updated', 'Android_Ver']], axis=1)

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows. 

In [111]:
df_apps.isna().any()

App               False
Category          False
Rating             True
Reviews           False
Size_MBs          False
Installs          False
Type               True
Price             False
Content_Rating    False
Genres            False
dtype: bool

In [12]:
rating_row_has_nan = df_apps[df_apps.Rating.isna()]

In [13]:
print(rating_row_has_nan.shape)

(1474, 10)


In [14]:
df_apps_clean = df_apps.dropna()

In [15]:
df_apps_clean.shape

(9367, 10)

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`. 


In [16]:
df_apps_clean.duplicated()

21       False
28       False
47       False
82       False
99       False
         ...  
10836    False
10837    False
10838    False
10839     True
10840    False
Length: 9367, dtype: bool

In [17]:
df_apps_clean[df_apps_clean['App'] == 'Instagram'].duplicated().count()

4

In [18]:
df_apps_clean.drop_duplicates(inplace=True)

C:\Users\jbk\AppData\Local\Temp\ipykernel_11252\3608147473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_apps_clean.drop_duplicates(inplace=True)


In [19]:
df_apps_clean.shape

(8891, 10)

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [20]:
df_apps_clean.describe()

,Rating,Reviews,Size_MBs
count,"8,891.00","8,891.00","8,891.00"
mean,4.19,"472,806.76",20.71
std,0.52,"2,905,214.05",22.18
min,1.00,1.00,0.01
25%,4.00,164.00,5.10
50%,4.30,"4,706.00",12.00
75%,4.50,"71,128.50",29.00
max,5.00,"78,158,306.00",100.00


In [112]:
df_apps_clean[df_apps_clean['Rating'] == 5]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_Estimate
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0.00,Everyone,Medical,0.00
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,1.49,Everyone,Arcade,1.49
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,0.99,Everyone,Arcade,0.99
82,Brick Breaker BR,GAME,5.00,7,19.00,5,Free,0.00,Everyone,Arcade,0.00
99,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.00,1,4.60,5,Free,0.00,Everyone,Medical,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2680,Florida Wildflowers,FAMILY,5.00,5,69.00,1000,Free,0.00,Everyone,Education,0.00
2750,"Superheroes, Marvel, DC, Comics, TV, Movies News",COMICS,5.00,34,12.00,5000,Free,0.00,Everyone,Comics,0.00
3030,CL Keyboard - Myanmar Keyboard (No Ads),TOOLS,5.00,24,3.20,5000,Free,0.00,Everyone,Tools,0.00
3115,Oración CX,LIFESTYLE,5.00,103,3.80,5000,Free,0.00,Everyone,Lifestyle,0.00


In [22]:
print(f"Rating alone may not determine an app is good since you don't really know the average rating of each app. An app could have rated by one person with a highest score of 5 in this case, and nobody else had ever ratedthe app again, but the app could be shown having highest rating score still -- this means this app isn't goodor popular enough to have enough people rate it -- but based on rating alone we don't know that.")

Rating alone may not determine an app is good since you don't really know the average rating of each app. An app could have rated by one person with a highest score of 5 in this case, and nobody else had ever ratedthe app again, but the app could be shown having highest rating score still -- this means this app isn't goodor popular enough to have enough people rate it -- but based on rating alone we don't know that.


# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please? 

In [23]:
df_apps_clean[df_apps_clean['Size_MBs'] == df_apps_clean['Size_MBs'].max()].head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
1795,Navi Radiography Pro,MEDICAL,4.70,11,100.00,500,Paid,$15.99,Everyone,Medical
3144,Vi Trainer,HEALTH_AND_FITNESS,3.60,124,100.00,"5,000",Free,0,Everyone,Health & Fitness
4176,Car Crash III Beam DH Real Damage Simulator 2018,GAME,3.60,151,100.00,"10,000",Free,0,Everyone,Racing
7926,Post Bank,FINANCE,4.50,60449,100.00,"1,000,000",Free,0,Everyone,Finance
7927,The Walking Dead: Our World,GAME,4.00,22435,100.00,"1,000,000",Free,0,Teen,Action


In [24]:
print(f'It sure looks like there is a cap on how big an app could be.')

It sure looks like there is a cap on how big an app could be.


# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [25]:
df_apps_clean.sort_values(by=['Reviews'], ascending=True).tail().iloc[::-1]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10811,Facebook,SOCIAL,4.10,78128208,5.30,"1,000,000,000",Free,0,Teen,Social
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication
10797,WhatsApp Messenger,COMMUNICATION,4.40,69109672,3.50,"1,000,000,000",Free,0,Everyone,Communication
10808,Instagram,SOCIAL,4.50,66577446,5.30,"1,000,000,000",Free,0,Teen,Social


In [26]:
top_50_most_reviewed_apps = df_apps_clean.sort_values(by=['Reviews'], ascending=True).tail(50).iloc[::-1]

In [27]:
top_50_most_reviewed_apps[top_50_most_reviewed_apps['Type'] == 'Paid']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres


In [28]:
print(f"There isn't seem to be any paid app that is in the top 50 most reviewed apps.")

There isn't seem to be any paid app that is in the top 50 most reviewed apps.


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [29]:
ratings = df_apps_clean['Content_Rating'].value_counts()

In [30]:
ratings

Everyone           7094
Teen               1022
Mature 17+          411
Everyone 10+        360
Adults only 18+       3
Unrated               1
Name: Content_Rating, dtype: int64

In [31]:
new_ratings = ratings.to_numpy()

In [32]:
print(new_ratings)

[7094 1022  411  360    3    1]


In [33]:
rating_names = ratings.keys()

In [34]:
fig = px.pie(labels=new_ratings, values=ratings.values, title="Content Rating", names=rating_names)

In [115]:
fig.update_traces(textposition='outside', textinfo='percent+label')

In [36]:
fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install? 

Check the datatype of the Installs column.

Count the number of apps at each level of installations. 

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first. 

In [37]:
df_apps_clean['Installs'] = df_apps_clean['Installs'].str.replace(',', '').astype(int)

C:\Users\jbk\AppData\Local\Temp\ipykernel_11252\1875258916.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
df_apps_clean[df_apps_clean['Installs'] >= 1000000000].count()

App               49
Category          49
Rating            49
Reviews           49
Size_MBs          49
Installs          49
Type              49
Price             49
Content_Rating    49
Genres            49
dtype: int64

In [39]:
df_apps_clean[['App', 'Installs']].groupby('Installs').count()

,App
Installs,
1,3
5,9
10,69
50,56
100,303
500,199
1000,699
5000,426
10000,989


In [40]:
print('It seems that there are fortynine apps that are over 1 billion installs.')

It seems that there are fortynine apps that are over 1 billion installs.


In [41]:
df_apps_clean[df_apps_clean['Installs'] == 1].count()

App               3
Category          3
Rating            3
Reviews           3
Size_MBs          3
Installs          3
Type              3
Price             3
Content_Rating    3
Genres            3
dtype: int64

In [42]:
print('It seems there are only three apps that got 1 install.')

It seems there are only three apps that got 1 install.


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [43]:
df_apps_clean['Price'] = df_apps_clean['Price'].str.replace('$', '').astype(float)

C:\Users\jbk\AppData\Local\Temp\ipykernel_11252\3963059841.py:1: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

C:\Users\jbk\AppData\Local\Temp\ipykernel_11252\3963059841.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
most_20_expensive_prices = df_apps_clean['Price'].sort_values().tail(20).iloc[::-1]
most_20_expensive_prices_list = list(most_20_expensive_prices.index)
print(most_20_expensive_prices_list)
df_most_20_expensive_apps = df_apps_clean[df_apps_clean.index.isin(most_20_expensive_prices_list)].sort_values(by=['Price']).iloc[::-1]
df_most_20_expensive_apps



[3946, 3145, 4606, 2775, 3114, 2394, 1946, 2461, 3221, 3554, 1331, 5765, 3897, 2193, 3856, 2281, 1407, 2629, 2481, 4264]


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
3946,I'm Rich - Trump Edition,LIFESTYLE,3.60,275,7.30,10000,Paid,400.00,Everyone,Lifestyle
5765,I am rich,LIFESTYLE,3.80,3547,1.80,100000,Paid,399.99,Everyone,Lifestyle
3145,I am rich(premium),FINANCE,3.50,472,0.94,5000,Paid,399.99,Everyone,Finance
1946,I am rich (Most expensive app),FINANCE,4.10,129,2.70,1000,Paid,399.99,Teen,Finance
2394,I am Rich!,FINANCE,3.80,93,22.00,1000,Paid,399.99,Everyone,Finance
2461,I AM RICH PRO PLUS,FINANCE,4.00,36,41.00,1000,Paid,399.99,Everyone,Finance
4606,I Am Rich Premium,FINANCE,4.10,1867,4.70,50000,Paid,399.99,Everyone,Finance
3114,I am Rich,FINANCE,4.30,180,3.80,5000,Paid,399.99,Everyone,Finance
2775,I Am Rich Pro,FAMILY,4.40,201,2.70,5000,Paid,399.99,Everyone,Entertainment
3221,I am Rich Plus,FAMILY,4.00,856,8.70,10000,Paid,399.99,Everyone,Entertainment


In [45]:
df_apps_clean = df_apps_clean.drop(df_apps_clean[df_apps_clean['Price'] > 250.00].index)

In [46]:
df_apps_clean.shape

(8876, 10)

In [47]:
df_revenue_estimate = df_apps_clean.Price * df_apps_clean.Installs

In [48]:
df_revenue_estimate.iloc[26]

149.9

In [49]:
df_apps_clean.iloc[26]

App               Chess of Blades (BL/Yaoi Game) (No VA)
Category                                          FAMILY
Rating                                              4.80
Reviews                                                4
Size_MBs                                           23.00
Installs                                              10
Type                                                Paid
Price                                              14.99
Content_Rating                                      Teen
Genres                                            Casual
Name: 233, dtype: object

In [50]:
df_apps_clean['Revenue_Estimate'] = df_revenue_estimate

In [51]:
df_apps_clean.sort_values(by='Revenue_Estimate').tail(10).iloc[::-1]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_Estimate
9224,Minecraft,FAMILY,4.50,2375336,19.00,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,"69,900,000.00"
9220,Minecraft,FAMILY,4.50,2376564,19.00,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,"69,900,000.00"
8825,Hitman Sniper,GAME,4.60,408292,29.00,10000000,Paid,0.99,Mature 17+,Action,"9,900,000.00"
7151,Grand Theft Auto: San Andreas,GAME,4.40,348962,26.00,1000000,Paid,6.99,Mature 17+,Action,"6,990,000.00"
7477,Facetune - For Free,PHOTOGRAPHY,4.40,49553,48.00,1000000,Paid,5.99,Everyone,Photography,"5,990,000.00"
7977,Sleep as Android Unlock,LIFESTYLE,4.50,23966,0.85,1000000,Paid,5.99,Everyone,Lifestyle,"5,990,000.00"
6594,DraStic DS Emulator,GAME,4.60,87766,12.00,1000000,Paid,4.99,Everyone,Action,"4,990,000.00"
6082,Weather Live,WEATHER,4.50,76593,4.75,500000,Paid,5.99,Everyone,Weather,"2,995,000.00"
6856,Threema,COMMUNICATION,4.50,51110,3.50,1000000,Paid,2.99,Everyone,Communication,"2,990,000.00"
7633,Five Nights at Freddy's,GAME,4.60,100805,50.00,1000000,Paid,2.99,Teen,Action,"2,990,000.00"


In [52]:
highest_10_paid_gross_apps_cats = df_apps_clean.sort_values(by='Revenue_Estimate').tail(10).iloc[::-1][['Category', 'Revenue_Estimate']]

In [53]:
highest_10_paid_gross_apps_cats_values = highest_10_paid_gross_apps_cats.Revenue_Estimate
highest_10_paid_gross_apps_cats_values

9224   69,900,000.00
9220   69,900,000.00
8825    9,900,000.00
7151    6,990,000.00
7477    5,990,000.00
7977    5,990,000.00
6594    4,990,000.00
6082    2,995,000.00
6856    2,990,000.00
7633    2,990,000.00
Name: Revenue_Estimate, dtype: float64

In [54]:
highest_10_paid_gross_apps_cats_names = highest_10_paid_gross_apps_cats.Category
highest_10_paid_gross_apps_cats_names

9224           FAMILY
9220           FAMILY
8825             GAME
7151             GAME
7477      PHOTOGRAPHY
7977        LIFESTYLE
6594             GAME
6082          WEATHER
6856    COMMUNICATION
7633             GAME
Name: Category, dtype: object

In [55]:
highest_10_paid_gross_apps_cats_labels = highest_10_paid_gross_apps_cats.to_numpy()

In [56]:
highest_10_paid_gross_apps_cats_labels

array([['FAMILY', 69900000.0],
       ['FAMILY', 69900000.0],
       ['GAME', 9900000.0],
       ['GAME', 6990000.0],
       ['PHOTOGRAPHY', 5990000.0],
       ['LIFESTYLE', 5990000.0],
       ['GAME', 4990000.0],
       ['WEATHER', 2995000.0],
       ['COMMUNICATION', 2990000.0],
       ['GAME', 2990000.0]], dtype=object)

In [57]:
fig_2 = px.pie(values=highest_10_paid_gross_apps_cats_values, labels=highest_10_paid_gross_apps_cats_labels, names=highest_10_paid_gross_apps_cats_names, title='Highest Category Gross of Paid Apps')

In [58]:
fig_2.update_traces(textposition='outside', textinfo='percent+label')

In [59]:
fig_2.show()

In [60]:
print('Based on the pie chart and other information, four apps that are games are among the highest paid gross apps.  This represents 13.6% of the top 10 highest paid gross apps.  At a closer look, Minecraft is also a game but it is being categorized as Family -- thus it seems the game category seems to be more popular than at a glance.')

Based on the pie chart and other information, four apps that are games are among the highest paid gross apps.  This represents 13.6% of the top 10 highest paid gross apps.  At a closer look, Minecraft is also a game but it is being categorized as Family -- thus it seems the game category seems to be more popular than at a glance.


### The most expensive apps sub $250

In [61]:
apps_less_than_price_of_250_df = df_apps_clean[df_apps_clean.Price < 250]
apps_less_than_price_of_250_df.sort_values(by='Price').iloc[::-1].head(50)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_Estimate
2281,Vargo Anesthesia Mega App,MEDICAL,4.60,92,32.00,1000,Paid,79.99,Everyone,Medical,"79,990.00"
1407,LTC AS Legal,MEDICAL,4.00,6,1.30,100,Paid,39.99,Everyone,Medical,"3,999.00"
2629,I am Rich Person,LIFESTYLE,4.20,134,1.80,1000,Paid,37.99,Everyone,Lifestyle,"37,990.00"
2481,A Manual of Acupuncture,MEDICAL,3.50,214,68.00,1000,Paid,33.99,Everyone,Medical,"33,990.00"
4264,Golfshot Plus: Golf GPS,SPORTS,4.10,3387,25.00,50000,Paid,29.99,Everyone,Sports,"1,499,500.00"
2207,EMT PASS,MEDICAL,3.40,51,2.40,1000,Paid,29.99,Everyone,Medical,"29,990.00"
2463,PTA Content Master,MEDICAL,4.20,64,41.00,1000,Paid,29.99,Everyone,Medical,"29,990.00"
504,AP Art History Flashcards,FAMILY,5.00,1,96.00,10,Paid,29.99,Mature 17+,Education,299.90
4774,Human Anatomy Atlas 2018: Complete 3D Human Body,MEDICAL,4.50,2923,25.00,100000,Paid,24.99,Everyone,Medical,"2,499,000.00"
4772,Human Anatomy Atlas 2018: Complete 3D Human Body,MEDICAL,4.50,2921,25.00,100000,Paid,24.99,Everyone,Medical,"2,499,000.00"


### Highest Grossing Paid Apps (ballpark estimate)

In [62]:
df_apps_clean.sort_values(by='Revenue_Estimate').iloc[::-1].head(50)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_Estimate
9224,Minecraft,FAMILY,4.50,2375336,19.00,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,"69,900,000.00"
9220,Minecraft,FAMILY,4.50,2376564,19.00,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,"69,900,000.00"
8825,Hitman Sniper,GAME,4.60,408292,29.00,10000000,Paid,0.99,Mature 17+,Action,"9,900,000.00"
7151,Grand Theft Auto: San Andreas,GAME,4.40,348962,26.00,1000000,Paid,6.99,Mature 17+,Action,"6,990,000.00"
7477,Facetune - For Free,PHOTOGRAPHY,4.40,49553,48.00,1000000,Paid,5.99,Everyone,Photography,"5,990,000.00"
7977,Sleep as Android Unlock,LIFESTYLE,4.50,23966,0.85,1000000,Paid,5.99,Everyone,Lifestyle,"5,990,000.00"
6594,DraStic DS Emulator,GAME,4.60,87766,12.00,1000000,Paid,4.99,Everyone,Action,"4,990,000.00"
6082,Weather Live,WEATHER,4.50,76593,4.75,500000,Paid,5.99,Everyone,Weather,"2,995,000.00"
6856,Threema,COMMUNICATION,4.50,51110,3.50,1000000,Paid,2.99,Everyone,Communication,"2,990,000.00"
7633,Five Nights at Freddy's,GAME,4.60,100805,50.00,1000000,Paid,2.99,Teen,Action,"2,990,000.00"


# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [63]:
df_apps_clean.Category.nunique()

33

In [64]:
top10_categories = df_apps_clean.Category.value_counts()[:10]

In [65]:
top10_categories

FAMILY             1714
GAME               1074
TOOLS               733
PRODUCTIVITY        334
FINANCE             311
PERSONALIZATION     310
COMMUNICATION       307
PHOTOGRAPHY         304
MEDICAL             302
LIFESTYLE           301
Name: Category, dtype: int64

In [66]:
px.bar(x=top10_categories.values, y=top10_categories.index)

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [67]:
px.bar(x=top10_categories.index, y=top10_categories.values)

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [68]:
category_installs = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum})

In [69]:
category_installs.sort_values(by='Installs', ascending=True, inplace=True)

In [70]:
category_installs.head()

,Installs
Category,
EVENTS,15949410
BEAUTY,26916200
PARENTING,31116110
MEDICAL,42162676
AUTO_AND_VEHICLES,53129800


In [71]:
fig_3 = px.bar(x=category_installs.Installs, y=category_installs.index)

In [72]:
fig_3.update_layout(title='Category Popularity')

In [73]:
fig_3.update_layout(xaxis_title='Numbers of Downloads', yaxis_title='Categories')

In [74]:
fig_3.show()

### Category Concentration - Downloads vs. Competition

**Challenge**: 
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this. 

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log') 

In [75]:
cat_number = df_apps_clean.groupby('Category').agg({'App': pd.Series.count})
cat_merge_df = pd.merge(cat_number, category_installs, on='Category', how='inner')

In [76]:
fig_4 = px.scatter(cat_merge_df, x='App', y='Installs', color=cat_merge_df.Installs, size='App',  hover_name=cat_merge_df.index)

In [77]:
fig_4.update_layout(title='Category Concentration', xaxis_title='Number of Apps(Lower=More Concentrated)',
                   yaxis=dict(type='log'))
fig_4.show()

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). 


In [78]:
print(f'There are {df_apps_clean.Genres.nunique()} types of genres')

There are 115 types of genres


In [79]:
genres_number = df_apps_clean.Genres.str.split(';', expand=True).stack()

In [80]:
genres_number = genres_number.value_counts()

In [81]:
number_of_genres = len(genres_number)

In [82]:
number_of_genres

53

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data? 

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/). 

* Find a way to set the colour scale using the color_continuous_scale parameter. 
* Find a way to make the color axis disappear by using coloraxis_showscale. 

In [83]:
fig_5 = px.bar(genres_number, color=genres_number.values, color_continuous_scale='Agsunset', x=genres_number.index, y=genres_number.values, title='Top Genres')

In [84]:
fig_5.update_layout(xaxis_title='Genre', yaxis_title='Number of Apps', xaxis = go.layout.XAxis(
        tickangle = 45), coloraxis_showscale=False)

In [85]:
fig_5.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [86]:
df_apps_clean.Type.value_counts()

Free    8278
Paid     598
Name: Type, dtype: int64

In [87]:
df_paid_vs_free = df_apps_clean.groupby(['Category', 'Type'], as_index=False).agg({'App': pd.Series.count})

In [88]:
df_paid_vs_free.head(25)

,Category,Type,App
0,ART_AND_DESIGN,Free,59
1,ART_AND_DESIGN,Paid,3
2,AUTO_AND_VEHICLES,Free,72
3,AUTO_AND_VEHICLES,Paid,1
4,BEAUTY,Free,42
5,BOOKS_AND_REFERENCE,Free,169
6,BOOKS_AND_REFERENCE,Paid,8
7,BUSINESS,Free,261
8,BUSINESS,Paid,9
9,COMICS,Free,58


**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart: 

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category. 

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value). 

In [89]:
fig_6 = px.bar(df_paid_vs_free, x='App', y='Category', color='Type', barmode='group', title='Free vs Paid Apps by Category', orientation='h')

In [90]:
fig_6.update_xaxes(categoryorder='total descending')

In [91]:
fig_6.update_layout(xaxis=dict(type='log'), autosize=False, width=900, height=900, xaxis_title='Number of Apps')

In [92]:
fig_6.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart. 

<img src=https://imgur.com/uVsECT3.png>


In [93]:
df_box_plots_paid_vs_free = df_apps_clean.groupby('Type', as_index=False).agg({'Installs': pd.Series.sum})
df_box_plots_paid_vs_free_median = df_apps_clean.groupby('Type', as_index=False).agg({'Installs': pd.Series.median})

In [94]:
df_box_plots_paid_vs_free

,Type,Installs
0,Free,146536739856
1,Paid,68993382


In [95]:
df_box_plots_paid_vs_free_median

,Type,Installs
0,Free,"500,000.00"
1,Paid,"5,000.00"


In [96]:
fig_7 = px.box(df_apps_clean, x='Type', y='Installs', color='Type', notched=True, points='all', title='How Many Downloads are Paid Apps Giving Up?')

In [97]:
fig_7.update_layout(yaxis=dict(type='log'))

In [98]:
fig_7.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below: 

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories. 

In [99]:
df_paid_apps = df_apps_clean[df_apps_clean['Type'] == 'Paid']

In [100]:
fig_8 = px.box(df_paid_apps, x='Category', y='Revenue_Estimate', title='How Much Can Paid Apps Earn?')

In [101]:
fig_8.update_layout(yaxis=dict(type='log'), xaxis={'categoryorder':'min ascending'}, xaxis_title='Category',
                  yaxis_title='Paid App Ballpark Revenue')

In [102]:
fig_8.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [103]:
df_box_plots_paid_median_price = df_apps_clean[df_apps_clean['Type'] == 'Paid'].agg({'Price': pd.Series.median})

In [104]:
median_price_of_paid_apps = str(df_box_plots_paid_median_price.values).strip('[').strip(']')

In [105]:
print(f'Median price of paid apps is ${float(median_price_of_paid_apps)}.')

Median price of paid apps is $2.99.


In [106]:
df_category_earning = df_apps_clean[df_apps_clean['Type'] == 'Paid']

In [107]:
fig_9 = px.box(df_category_earning, x='Category', y='Price', title='How Much Can Each Category Earn?')

In [108]:
fig_9.update_layout(yaxis=dict(type='log'), xaxis={'categoryorder':'max descending'}, xaxis_title='Category',
                  yaxis_title='Paid App Price')

In [109]:
fig_9.show()